In [13]:
import pandas as pd
from keras.layers import Dense,Dropout,Activation
from keras.layers.normalization import BatchNormalization
# from keras.activations import relu
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

In [3]:
train_df = pd.read_csv('dataset/train_dataset.csv')
test_df = pd.read_csv('dataset/test_dataset.csv')

In [24]:
target = train_df['信用分']
data = pd.concat([train_df.drop(columns=['信用分']), test_df], axis=0, ignore_index=True)
data = data.drop(columns=['用户编码'])

In [25]:
from sklearn.preprocessing import robust_scale
data = pd.DataFrame(robust_scale(data.values), columns=data.columns)

In [27]:
train = data.loc[:49999, :]
test = data.loc[50000:, :]

In [28]:
drop_columns = [ '是否大学生客户',
                '用户实名制是否通过核实', '当月是否到过福州山姆会员店', 
                '当月是否逛过福州仓山万达']

X_train = train.drop(columns=drop_columns).values
y_train = target.values
X_test = test.drop(columns=drop_columns).values

In [35]:
y_train, bins = pd.cut(y_train, 30, retbins=True, labels=False)

In [33]:
class NNModels(object):
    def __init__(self):
        pass

    @staticmethod
    def _get_nn_base_model(input_dim=24):
        nn_model = Sequential()

        nn_model.add(Dense(50, input_dim=input_dim, kernel_initializer='normal'))
        nn_model.add(Activation('relu'))
        nn_model.add(BatchNormalization())
        nn_model.add(Dropout(0.5))

        nn_model.add(Dense(200, kernel_initializer='normal'))
        nn_model.add(Activation('relu'))
        nn_model.add(BatchNormalization())
        nn_model.add(Dropout(0.5))

        nn_model.add(Dense(200, kernel_initializer='normal'))
        nn_model.add(Activation('relu'))
        nn_model.add(BatchNormalization())
        nn_model.add(Dropout(0.5))

        nn_model.add(Dense(30, kernel_initializer='normal'))
        nn_model.compile(loss='mae', optimizer='adam', metrics=['mae'])

        return nn_model

    def nn_model(self, x_data, y_data, x_test):

        baseline_model = self._get_nn_base_model
        estimator = KerasRegressor(build_fn=baseline_model, epochs=1000, batch_size=100, verbose=1)

        # kfold = KFold(n_splits=5)
        # mae = make_scorer(mean_absolute_error)
        # res = cross_val_score(estimator, X=x_data, y=y_data, cv=kfold, scoring=mae)
        # mae_error = np.mean(res)
        estimator.fit(x_data, y_data)
        y_pred = estimator.predict(x_data)
        mae_error = mean_absolute_error(y_pred, y_data)

        print(f'mae error: {mae_error}')
        print(f'nn score: {1 / (1 + mae_error)}')

        pred = estimator.predict(x_test)
        sub_df = pd.read_csv('dataset/submit_example.csv')
        sub_df[' score'] = np.rint(pred).astype(int)
        sub_df.to_csv("output/{}.csv".format(model_name), index=False)

In [34]:
nn = NNModels()
nn.nn_model(X_train, y_train, X_test)

Epoch 1/1000
50000/50000 [==============================] - 15s 293us/step - loss: 602.0990 - mean_absolute_error: 602.0990
Epoch 2/1000
50000/50000 [==============================] - 10s 200us/step - loss: 482.8804 - mean_absolute_error: 482.8804
Epoch 3/1000
50000/50000 [==============================] - 10s 195us/step - loss: 248.2682 - mean_absolute_error: 248.2682
Epoch 4/1000
50000/50000 [==============================] - 10s 196us/step - loss: 63.4830 - mean_absolute_error: 63.4830
Epoch 5/1000
50000/50000 [==============================] - 10s 196us/step - loss: 55.4905 - mean_absolute_error: 55.4905
Epoch 6/1000
50000/50000 [==============================] - 10s 196us/step - loss: 54.4449 - mean_absolute_error: 54.4449
Epoch 7/1000
50000/50000 [==============================] - 10s 198us/step - loss: 54.2005 - mean_absolute_error: 54.2005
Epoch 8/1000
50000/50000 [==============================] - 10s 199us/step - loss: 54.2984 - mean_absolute_error: 54.2984
Epoch 9/1000
50000

50000/50000 [==============================] - 10s 194us/step - loss: 51.2466 - mean_absolute_error: 51.2466
Epoch 68/1000
50000/50000 [==============================] - 10s 195us/step - loss: 51.1516 - mean_absolute_error: 51.1516
Epoch 69/1000
50000/50000 [==============================] - 10s 194us/step - loss: 51.0331 - mean_absolute_error: 51.0331
Epoch 70/1000
50000/50000 [==============================] - 10s 194us/step - loss: 50.8199 - mean_absolute_error: 50.8199
Epoch 71/1000
50000/50000 [==============================] - 10s 195us/step - loss: 51.0088 - mean_absolute_error: 51.0088
Epoch 72/1000
50000/50000 [==============================] - 10s 194us/step - loss: 50.9308 - mean_absolute_error: 50.9308
Epoch 73/1000
50000/50000 [==============================] - 10s 194us/step - loss: 50.7920 - mean_absolute_error: 50.7920
Epoch 74/1000
50000/50000 [==============================] - 10s 194us/step - loss: 50.9545 - mean_absolute_error: 50.9545
Epoch 75/1000
50000/50000 [===

50000/50000 [==============================] - 10s 194us/step - loss: 49.5792 - mean_absolute_error: 49.5792
Epoch 133/1000
50000/50000 [==============================] - 10s 194us/step - loss: 49.3527 - mean_absolute_error: 49.3527
Epoch 134/1000
50000/50000 [==============================] - 10s 195us/step - loss: 49.6228 - mean_absolute_error: 49.6228
Epoch 135/1000
50000/50000 [==============================] - 10s 194us/step - loss: 49.5862 - mean_absolute_error: 49.5862
Epoch 136/1000
50000/50000 [==============================] - 10s 194us/step - loss: 49.3677 - mean_absolute_error: 49.3677
Epoch 137/1000
50000/50000 [==============================] - 10s 194us/step - loss: 49.5971 - mean_absolute_error: 49.5971
Epoch 138/1000
50000/50000 [==============================] - 10s 197us/step - loss: 49.2138 - mean_absolute_error: 49.2138
Epoch 139/1000
50000/50000 [==============================] - 10s 196us/step - loss: 49.3064 - mean_absolute_error: 49.3064
Epoch 140/1000
50000/50

50000/50000 [==============================] - 10s 195us/step - loss: 48.1770 - mean_absolute_error: 48.1770
Epoch 198/1000
50000/50000 [==============================] - 10s 195us/step - loss: 48.3095 - mean_absolute_error: 48.3095
Epoch 199/1000
50000/50000 [==============================] - 10s 194us/step - loss: 48.3488 - mean_absolute_error: 48.3488
Epoch 200/1000
50000/50000 [==============================] - ETA: 0s - loss: 48.0190 - mean_absolute_error: 48.019 - 10s 194us/step - loss: 48.0280 - mean_absolute_error: 48.0280
Epoch 201/1000
50000/50000 [==============================] - 10s 195us/step - loss: 48.0096 - mean_absolute_error: 48.0096
Epoch 202/1000
50000/50000 [==============================] - 10s 195us/step - loss: 47.8029 - mean_absolute_error: 47.8029
Epoch 203/1000
50000/50000 [==============================] - 10s 195us/step - loss: 48.0720 - mean_absolute_error: 48.0720
Epoch 204/1000
50000/50000 [==============================] - 10s 195us/step - loss: 48.059

50000/50000 [==============================] - 10s 194us/step - loss: 46.6427 - mean_absolute_error: 46.6427
Epoch 263/1000
50000/50000 [==============================] - 10s 195us/step - loss: 46.8640 - mean_absolute_error: 46.8640
Epoch 264/1000
50000/50000 [==============================] - 10s 195us/step - loss: 46.5723 - mean_absolute_error: 46.5723
Epoch 265/1000
50000/50000 [==============================] - 10s 194us/step - loss: 46.8795 - mean_absolute_error: 46.8795
Epoch 266/1000
50000/50000 [==============================] - 10s 194us/step - loss: 46.7001 - mean_absolute_error: 46.7001
Epoch 267/1000
50000/50000 [==============================] - 10s 194us/step - loss: 46.6863 - mean_absolute_error: 46.6863
Epoch 268/1000
50000/50000 [==============================] - 10s 194us/step - loss: 46.6869 - mean_absolute_error: 46.6869
Epoch 269/1000
50000/50000 [==============================] - 10s 194us/step - loss: 46.8047 - mean_absolute_error: 46.8047
Epoch 270/1000
50000/50

...

50000/50000 [==============================] - 10s 195us/step - loss: 36.8268 - mean_absolute_error: 36.8268
Epoch 781/1000
50000/50000 [==============================] - 10s 194us/step - loss: 36.8295 - mean_absolute_error: 36.8295
Epoch 782/1000
50000/50000 [==============================] - 10s 194us/step - loss: 36.8372 - mean_absolute_error: 36.8372
Epoch 783/1000
50000/50000 [==============================] - 10s 201us/step - loss: 36.6753 - mean_absolute_error: 36.6753
Epoch 784/1000
50000/50000 [==============================] - 10s 199us/step - loss: 36.4441 - mean_absolute_error: 36.4441
Epoch 785/1000
50000/50000 [==============================] - 10s 194us/step - loss: 36.9154 - mean_absolute_error: 36.9154
Epoch 786/1000
50000/50000 [==============================] - 10s 195us/step - loss: 36.5773 - mean_absolute_error: 36.5773
Epoch 787/1000
50000/50000 [==============================] - 10s 195us/step - loss: 36.8425 - mean_absolute_error: 36.8425
Epoch 788/1000
50000/50

50000/50000 [==============================] - 10s 198us/step - loss: 35.4487 - mean_absolute_error: 35.4487
Epoch 847/1000
50000/50000 [==============================] - 10s 196us/step - loss: 35.6530 - mean_absolute_error: 35.6530
Epoch 848/1000
50000/50000 [==============================] - 10s 194us/step - loss: 35.5997 - mean_absolute_error: 35.5997
Epoch 849/1000
50000/50000 [==============================] - 10s 195us/step - loss: 35.5015 - mean_absolute_error: 35.5015
Epoch 850/1000
50000/50000 [==============================] - 10s 194us/step - loss: 35.3779 - mean_absolute_error: 35.3779
Epoch 851/1000
50000/50000 [==============================] - 10s 194us/step - loss: 35.4816 - mean_absolute_error: 35.4816
Epoch 852/1000
50000/50000 [==============================] - 10s 195us/step - loss: 35.5305 - mean_absolute_error: 35.5305
Epoch 853/1000
50000/50000 [==============================] - 10s 196us/step - loss: 35.2756 - mean_absolute_error: 35.2756
Epoch 854/1000
50000/50

50000/50000 [==============================] - 10s 194us/step - loss: 34.4235 - mean_absolute_error: 34.4235
Epoch 912/1000
50000/50000 [==============================] - 10s 195us/step - loss: 34.5694 - mean_absolute_error: 34.5694
Epoch 913/1000
50000/50000 [==============================] - 10s 195us/step - loss: 34.2766 - mean_absolute_error: 34.2766
Epoch 914/1000
50000/50000 [==============================] - 10s 196us/step - loss: 34.7423 - mean_absolute_error: 34.7423
Epoch 915/1000
50000/50000 [==============================] - 10s 194us/step - loss: 34.4348 - mean_absolute_error: 34.4348
Epoch 916/1000
50000/50000 [==============================] - 10s 194us/step - loss: 34.3780 - mean_absolute_error: 34.3780
Epoch 917/1000
50000/50000 [==============================] - 10s 194us/step - loss: 34.3981 - mean_absolute_error: 34.3981
Epoch 918/1000
50000/50000 [==============================] - 10s 195us/step - loss: 34.2764 - mean_absolute_error: 34.2764
Epoch 919/1000
50000/50

50000/50000 [==============================] - 10s 194us/step - loss: 33.1098 - mean_absolute_error: 33.1098
Epoch 977/1000
50000/50000 [==============================] - 10s 194us/step - loss: 33.3689 - mean_absolute_error: 33.3689
Epoch 978/1000
50000/50000 [==============================] - 10s 194us/step - loss: 33.1909 - mean_absolute_error: 33.1909
Epoch 979/1000
50000/50000 [==============================] - 10s 194us/step - loss: 33.2010 - mean_absolute_error: 33.2010
Epoch 980/1000
50000/50000 [==============================] - 10s 194us/step - loss: 33.3477 - mean_absolute_error: 33.3477
Epoch 981/1000
50000/50000 [==============================] - 10s 195us/step - loss: 33.2439 - mean_absolute_error: 33.2439
Epoch 982/1000
50000/50000 [==============================] - 10s 195us/step - loss: 33.1831 - mean_absolute_error: 33.1831
Epoch 983/1000
50000/50000 [==============================] - 10s 194us/step - loss: 33.1791 - mean_absolute_error: 33.1791
Epoch 984/1000
50000/50

NameError: name 'np' is not defined